In [1]:
import pandas as pd
import numpy as np
import multiprocessing
import gc

import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import warnings
import sklearn
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score, roc_curve, auc, classification_report, confusion_matrix, roc_auc_score
#warnings.filterwarnings('ignore')

## EDA 

In [2]:
data_path = '/kaggle/input/ieee-fraud-detection/'

# Load the data
train_transaction = pd.read_csv(f'{data_path}train_transaction.csv')
train_identity = pd.read_csv(f'{data_path}train_identity.csv')
test_transaction = pd.read_csv(f'{data_path}test_transaction.csv')
test_identity = pd.read_csv(f'{data_path}test_identity.csv')

print(f"\nTrain Transaction: {train_transaction.shape}")
print(f"Train Identity: {train_identity.shape}")
print(f"Test Transaction: {test_transaction.shape}")
print(f"Test Identity: {test_identity.shape}")


Train Transaction: (590540, 394)
Train Identity: (144233, 41)
Test Transaction: (506691, 393)
Test Identity: (141907, 41)


In [3]:
pd.set_option("display.max_columns", 400)
pd.set_option("display.max_rows", 500)

pd.options.display.max_columns
pd.options.display.max_rows

train_transaction.head(10)

,TransactionID,isFraud,TransactionDT,TransactionAmt,ProductCD,card1,card2,card3,card4,card5,card6,addr1,addr2,dist1,dist2,P_emaildomain,R_emaildomain,C1,C2,C3,C4,C5,C6,C7,C8,C9,C10,C11,C12,C13,C14,D1,D2,D3,D4,D5,D6,D7,D8,D9,D10,D11,D12,D13,D14,D15,M1,M2,M3,M4,M5,M6,M7,M8,M9,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13,V14,V15,V16,V17,V18,V19,V20,V21,V22,V23,V24,V25,V26,V27,V28,V29,V30,V31,V32,V33,V34,V35,V36,V37,V38,V39,V40,V41,V42,V43,V44,V45,V46,V47,V48,V49,V50,V51,V52,V53,V54,V55,V56,V57,V58,V59,V60,V61,V62,V63,V64,V65,V66,V67,V68,V69,V70,V71,V72,V73,V74,V75,V76,V77,V78,V79,V80,V81,V82,V83,V84,V85,V86,V87,V88,V89,V90,V91,V92,V93,V94,V95,V96,V97,V98,V99,V100,V101,V102,V103,V104,V105,V106,V107,V108,V109,V110,V111,V112,V113,V114,V115,V116,V117,V118,V119,V120,V121,V122,V123,V124,V125,V126,V127,V128,V129,V130,V131,V132,V133,V134,V135,V136,V137,V138,V139,V140,V141,V142,V143,V144,V145,V146,V147,V148,V149,V150,V151,V152,V153,V154,V155,V156,V157,V158,V159,V160,V161,V162,V163,V164,V165,V166,V167,V168,V169,V170,V171,V172,V173,V174,V175,V176,V177,V178,V179,V180,V181,V182,V183,V184,V185,V186,V187,V188,V189,V190,V191,V192,V193,V194,V195,V196,V197,V198,V199,V200,V201,V202,V203,V204,V205,V206,V207,V208,V209,V210,V211,V212,V213,V214,V215,V216,V217,V218,V219,V220,V221,V222,V223,V224,V225,V226,V227,V228,V229,V230,V231,V232,V233,V234,V235,V236,V237,V238,V239,V240,V241,V242,V243,V244,V245,V246,V247,V248,V249,V250,V251,V252,V253,V254,V255,V256,V257,V258,V259,V260,V261,V262,V263,V264,V265,V266,V267,V268,V269,V270,V271,V272,V273,V274,V275,V276,V277,V278,V279,V280,V281,V282,V283,V284,V285,V286,V287,V288,V289,V290,V291,V292,V293,V294,V295,V296,V297,V298,V299,V300,V301,V302,V303,V304,V305,V306,V307,V308,V309,V310,V311,V312,V313,V314,V315,V316,V317,V318,V319,V320,V321,V322,V323,V324,V325,V326,V327,V328,V329,V330,V331,V332,V333,V334,V335,V336,V337,V338,V339
0,2987000,0,86400,68.5,W,13926,NaN,150.0,discover,142.0,credit,315.0,87.0,19.0,NaN,NaN,NaN,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,2.0,0.0,1.0,1.0,14.0,NaN,13.0,NaN,NaN,NaN,NaN,NaN,NaN,13.0,13.0,NaN,NaN,NaN,0.0,T,T,T,M2,F,T,NaN,NaN,NaN,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,117.0,0.0,0.0,0.0,0.0,0.0,117.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,117.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,117.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2987001,0,86401,29.0,W,2755,404.0,150.0,mastercard,102.0,credit,325.0,87.0,NaN,NaN,gmail.com,NaN,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,M0,T,T,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,

In [4]:
train_transaction.dtypes.value_counts()
test_transaction.select_dtypes(include = 'object') 



,ProductCD,card4,card6,P_emaildomain,R_emaildomain,M1,M2,M3,M4,M5,M6,M7,M8,M9
0,W,visa,debit,gmail.com,NaN,T,T,F,NaN,NaN,F,T,T,T
1,W,visa,debit,aol.com,NaN,T,F,F,M0,NaN,F,NaN,NaN,NaN
2,W,visa,debit,hotmail.com,NaN,T,T,F,M0,F,F,F,F,F
3,W,visa,debit,gmail.com,NaN,T,T,T,NaN,NaN,T,NaN,NaN,NaN
4,W,mastercard,debit,gmail.com,NaN,T,T,T,NaN,NaN,F,F,T,T
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
506686,C,mastercard,debit,gmail.com,gmail.com,NaN,NaN,NaN,M2,NaN,NaN,NaN,NaN,NaN
506687,C,mastercard,debit,hotmail.com,hotmail.com,NaN,NaN,NaN,M2,NaN,NaN,NaN,NaN,NaN
506688,W,visa,debit,hotmail.com,NaN,T,T,T,M0,F,F,F,T,T
506689,W,mastercard,debit,hotmail.com,NaN,T,T,T,M0,F,F,F,F,F


In [5]:
train = pd.merge(train_transaction, train_identity, on='TransactionID', how='left')
test = pd.merge(test_transaction, test_identity, on='TransactionID', how='left')

del train_transaction, train_identity, test_transaction, test_identity
gc.collect();

In [6]:
train.shape

(590540, 434)

## Train Model

In [7]:
X = train.drop(['TransactionID','isFraud'], axis = 1)
y = train['isFraud']

In [8]:
y.head(10)

0    0
1    0
2    0
3    0
4    0
5    0
6    0
7    0
8    0
9    0
Name: isFraud, dtype: int64

In [9]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)

In [10]:
from sklearn.preprocessing import LabelEncoder

# Encode everything
encoders = {}

X_train_encoded = X_train.copy()
X_test_encoded = X_test.copy()

cat_columns = X.select_dtypes(include=['object']).columns
for col in cat_columns:
    le = LabelEncoder()
    le.fit(X_train[col].astype(str))
# Save encoder
    encoders[col] = le
 # Transform training
    X_train_encoded[col] = le.transform(X_train[col].astype(str))
    
# Transform validation (vectorised)
    train_classes = set(le.classes_)
    test_col = X_test[col].astype(str)
    mask = test_col.isin(train_classes)
    X_test_encoded[col] = -1
    X_test_encoded.loc[mask, col] = le.transform(test_col[mask])

In [11]:
X_test_encoded.dtypes.value_counts()

float64    399
int64       33
Name: count, dtype: int64

In [12]:

# Run RF on ALL 400 columns
rf_initial = RandomForestClassifier(n_estimators=100, max_depth=10, random_state=42, n_jobs=-1)
rf_initial.fit(X_train_encoded, y_train)

# Get top features
feature_importance = pd.DataFrame({
    'feature': X_train_encoded.columns,
    'importance': rf_initial.feature_importances_
}).sort_values('importance', ascending=False)

# Select top K features (e.g., top 50)
top_features = feature_importance.head(50)['feature'].tolist()


In [13]:
# Final model on selected features
X_train_selected = X_train_encoded[top_features]
X_test_selected = X_test_encoded[top_features]

rf_final = RandomForestClassifier(n_estimators=200, max_depth=15, random_state=42)
rf_final.fit(X_train_selected, y_train)

RandomForestClassifier(max_depth=15, n_estimators=200, random_state=42)

In [14]:
#y_pred = rf_initial.predict(X_test_encoded)
#y_pred_proba = rf_initial.predict_proba(X_test_encoded)[:, 1]

In [15]:
y_pred = rf_final.predict(X_test_selected)
y_pred_proba = rf_final.predict_proba(X_test_selected)[:, 1]

In [16]:
# Evaluate baseline


print(f"ROC-AUC Score: {roc_auc_score(y_test, y_pred_proba):.4f}")
print("\nClassification Report:")
print(classification_report(y_test, y_pred))
print("\nConfusion Matrix:")
print(confusion_matrix(y_test, y_pred))

ROC-AUC Score: 0.8806

Classification Report:
              precision    recall  f1-score   support

           0       0.98      1.00      0.99    113866
           1       0.93      0.33      0.49      4242

    accuracy                           0.98    118108
   macro avg       0.95      0.67      0.74    118108
weighted avg       0.97      0.98      0.97    118108


Confusion Matrix:
[[113759    107]
 [  2835   1407]]


## Prediction on Test set

In [17]:
test_ids = test['TransactionID']
test_features = test.drop('TransactionID', axis=1)


In [18]:


# 2. Encode the same way as training
test_encoded = test_features.copy()
#cat_columns = X_train.select_dtypes(include=['object']).columns

for col in cat_columns:
    if col in test_encoded.columns and col in encoders:
        le = encoders[col]  # Reuse
        # Vectorised encoding with unknown handling

        test_col = test_features[col].astype(str)
        train_classes = set(le.classes_)
        mask = test_col.isin(train_classes)
        test_encoded[col] = -1
        test_encoded.loc[mask, col] = le.transform(test_col[mask])

# 3. Select features (if using reduced model)
test_selected = test_encoded[top_features]

In [19]:

# 4. Predict
predictions = rf_final.predict_proba(test_selected)[:, 1]

# 5. Create submission
submission = pd.DataFrame({
    'TransactionID': test['TransactionID'],
    'isFraud': predictions
})

submission.to_csv('submission.csv', index=False)
print("Submission file created!")

Submission file created!
